
# 08: Parameter Sweep（研究用）

このノートは **research notebook** です。  
本番パラメータ確定ではなく、感度分析で「どの条件で壊れやすいか」を確認します。

対象:
- EMA_FAST / EMA_SLOW の組
- ATR_PERIOD
- REGIME_WIN



## 1) 評価方針

- シグナル: EMA cross + ATR regime
- リーク回避: `signal_t` に対して `return_{t->t+1}` を使用
- 指標:
  - `sharpe_like = mean/std*sqrt(252)`
  - `expectancy = E[r]`（1バーあたり平均）

> 注意: これは単純化した比較であり、手数料・スリッページ・約定制約は未考慮です。


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

from quantlab.indicators import ema, atr
from quantlab.backtest import (
    generate_positions_from_signals,
    compute_strategy_returns,
    summarize_performance,
)

plt.style.use("seaborn-v0_8")
pd.set_option("display.float_format", lambda x: f"{x:,.5f}")


In [ ]:

SYMBOL = "QQQ"
PERIOD = "8y"
EMA_PAIRS = [(8, 21), (12, 26), (20, 50)]
ATR_PERIODS = [10, 14, 20]
REGIME_WINS = [40, 60, 90]

df = yf.download(SYMBOL, period=PERIOD, auto_adjust=False, progress=False)
if isinstance(df.columns, pd.MultiIndex):
    df.columns = [c[0] for c in df.columns]
df = df[["Open", "High", "Low", "Close", "Volume"]].dropna().copy()
print(f"rows={len(df)}")


In [ ]:

def make_signal_ema_atr(frame: pd.DataFrame, ema_fast: int, ema_slow: int, atr_period: int, regime_win: int) -> pd.Series:
    """EMAクロス + ATRレジームのシグナル。"""
    x = frame.copy()
    x["ema_fast"] = ema(x["Close"], ema_fast)
    x["ema_slow"] = ema(x["Close"], ema_slow)
    x["ema_diff"] = x["ema_fast"] - x["ema_slow"]
    x["atr"] = atr(x, atr_period)
    x["atr_thresh"] = x["atr"].rolling(regime_win).median()

    prev = x["ema_diff"].shift(1)
    curr = x["ema_diff"]
    buy = (prev <= 0) & (curr > 0)
    sell = (prev >= 0) & (curr < 0)
    active = x["atr"] > x["atr_thresh"]

    sig = pd.Series("HOLD", index=x.index, dtype="object")
    sig.loc[buy & active] = "BUY"
    sig.loc[sell & active] = "SELL"
    return sig


def evaluate_combo(frame: pd.DataFrame, ema_fast: int, ema_slow: int, atr_period: int, regime_win: int) -> pd.Series:
    local = frame.copy()
    local["signal"] = make_signal_ema_atr(local, ema_fast, ema_slow, atr_period, regime_win)
    pos = generate_positions_from_signals(local)
    rets = compute_strategy_returns(local, pos)
    s = summarize_performance(rets)
    s["ema_fast"] = ema_fast
    s["ema_slow"] = ema_slow
    s["atr_period"] = atr_period
    s["regime_win"] = regime_win
    return s


In [ ]:

rows = []
for ef, es in EMA_PAIRS:
    for ap in ATR_PERIODS:
        for rw in REGIME_WINS:
            rows.append(evaluate_combo(df, ef, es, ap, rw))

result_df = pd.DataFrame(rows)
result_df = result_df.sort_values("sharpe_like", ascending=False).reset_index(drop=True)
print("Top 10 by sharpe_like")
print(result_df[["ema_fast", "ema_slow", "atr_period", "regime_win", "expectancy", "sharpe_like", "max_drawdown"]].head(10).round(5))


## 2) Heatmap（EMAペアごとに可視化）

In [ ]:

def plot_heatmaps(metric: str) -> None:
    n = len(EMA_PAIRS)
    fig, axes = plt.subplots(1, n, figsize=(5*n, 4), sharey=True)
    if n == 1:
        axes = [axes]

    for ax, (ef, es) in zip(axes, EMA_PAIRS):
        sub = result_df[(result_df["ema_fast"] == ef) & (result_df["ema_slow"] == es)]
        pivot = sub.pivot(index="atr_period", columns="regime_win", values=metric)

        im = ax.imshow(pivot.values, aspect="auto", origin="lower", cmap="viridis")
        ax.set_title(f"EMA({ef},{es}) - {metric}")
        ax.set_xlabel("regime_win")
        ax.set_ylabel("atr_period")
        ax.set_xticks(range(len(pivot.columns)))
        ax.set_xticklabels(pivot.columns)
        ax.set_yticks(range(len(pivot.index)))
        ax.set_yticklabels(pivot.index)

        # 値をセル内表示して、視覚 + 数値で確認できるようにする。
        for i in range(pivot.shape[0]):
            for j in range(pivot.shape[1]):
                v = pivot.values[i, j]
                ax.text(j, i, f"{v:.2f}", ha="center", va="center", color="white", fontsize=8)

        fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    fig.suptitle(f"Parameter Sweep Heatmaps ({metric})", y=1.02)
    plt.tight_layout()
    plt.show()


plot_heatmaps("sharpe_like")
plot_heatmaps("expectancy")



## 3) What to observe（観察ポイント）

- EMAペアが変わると、最適なATR/Regime窓は安定するか？
- `sharpe_like` と `expectancy` の最適点が一致しない場合、何を重視するか？
- 上位条件の差が僅差なら、より単純なパラメータを優先できるか？
- 期間外データ（out-of-sample）で再現するか？

---

### Reminder
この結果は **研究探索** です。実運用前には、
- 取引コスト込み評価
- ウォークフォワード
- 複数銘柄/複数期間での頑健性確認
を行ってください。
